<a href="https://colab.research.google.com/github/julianolm/my-torch-utils/blob/main/ArquiteturaBifurcada1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primeira versao da arquitetura bifurcada

Resnet18 (pretreinada/fixa) + LBCNNResnet18

In [ ]:
import torch, torchvision, torch.nn as nn
from torch import Tensor
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Resnet part

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

class Resnet(nn.Module):
    def __init__(self, pretrained: bool = True):
        super().__init__()
        self.net = torchvision.models.resnet18(pretrained=pretrained)
        for _, param in self.net.named_parameters():
            param.requires_grad = False
        self.net.avgpool = Identity()
        self.net.fc = Identity()
    
    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.net.conv1(x)
        x = self.net.bn1(x)
        x = self.net.relu(x)
        x = self.net.maxpool(x)

        x = self.net.layer1(x)
        x = self.net.layer2(x)
        x = self.net.layer3(x)
        x = self.net.layer4(x)

        x = self.net.avgpool(x)
        # x = torch.flatten(x, 1)
        x = self.net.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

resnet = Resnet()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
x = torch.randn(128, 3, 224, 224)

In [ ]:
resnet(x).shape

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([128, 512, 7, 7])

output was supposed to be (128, 512, 7, 7)

hitnt: 7x7x512 = 25088

In [ ]:
torchvision.models.resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## LBCNN part

In [ ]:
from drive.MyDrive.IC import juefei_lbcnn_resnet

In [ ]:
class LBCResnet(nn.Module):
    def __init__(self, pretrained: bool = False) -> None:
        super().__init__()
        self.net = juefei_lbcnn_resnet.resnet18(pretrained=pretrained)
        self.net.avgpool = Identity()
        self.net.fc = Identity()
    
    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.net.conv1(x)
        x = self.net.bn1(x)
        x = self.net.relu(x)
        x = self.net.maxpool(x)

        x = self.net.layer1(x)
        x = self.net.layer2(x)
        x = self.net.layer3(x)
        x = self.net.layer4(x)

        x = self.net.avgpool(x)
        # x = torch.flatten(x, 1)
        x = self.net.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

lbc_resnet = LBCResnet()

In [ ]:
lbc_resnet

LBCResnet(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): LBConvBN(
          (random_binary_conv): RandomBinaryConv()
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (fc): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        )
        (relu): ReLU(inplace=True)
        (conv2): LBConvBN(
          (random_binary_conv): RandomBinaryConv()
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (fc): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        )
      )
      (1): BasicBlock(
        (conv1): LBConvBN(
          (random_binary_con

In [ ]:
lbc_resnet(x).shape

torch.Size([128, 512, 7, 7])

##Mixed Model

In [ ]:
class MixedNet(nn.Module):
    def __init__(self):
        super(MixedNet, self).__init__()
        self.res = Resnet(pretrained=True)
        self.lbc = LBCResnet(pretrained=False)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.fc = nn.Linear(512, 10)

    
    def forward(self, x: Tensor) -> Tensor:
        x = self.res(x) + self.lbc(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [ ]:
mixed_net = MixedNet()

In [ ]:
mixed_net(x).shape

torch.Size([128, 10])

##Training

In [ ]:
import time
from torchvision import datasets, transforms

# import os
import sys
# import time
# import math

# import torch.nn.init as init

import torch.optim as optim
import torch.backends.cudnn as cudnn

# import torchvision
# from torchvision import datasets, transforms

# import argparse


# import numpy as np

# import torch
# import torch.nn as nn
# import torch.nn.functional as F

In [ ]:
# !unzip "/content/drive/MyDrive/fmd.zip"

In [ ]:
'''Some helper functions for PyTorch, including:
    - get_mean_and_std: calculate the mean and std value of dataset.
    - msr_init: net parameter initialization.
    - progress_bar: progress bar mimic xlua.progress.
'''



def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(3):
            mean[i] += inputs[:,i,:,:].mean()
            std[i] += inputs[:,i,:,:].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

def init_params(net):
    '''Init layer parameters.'''
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            init.kaiming_normal(m.weight, mode='fan_out')
            if m.bias:
                init.constant(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            init.constant(m.weight, 1)
            init.constant(m.bias, 0)
        elif isinstance(m, nn.Linear):
            init.normal(m.weight, std=1e-3)
            if m.bias:
                init.constant(m.bias, 0)


# _, term_width = os.popen('stty size', 'r').read().split()
# term_width = int(term_width)
term_width = 32

TOTAL_BAR_LENGTH = 65.
last_time = time.time()
begin_time = last_time
def progress_bar(current, total, msg=None):
    global last_time, begin_time
    if current == 0:
        begin_time = time.time()  # Reset for new bar.

    cur_len = int(TOTAL_BAR_LENGTH*current/total)
    rest_len = int(TOTAL_BAR_LENGTH - cur_len) - 1

    sys.stdout.write(' [')
    for i in range(cur_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    cur_time = time.time()
    step_time = cur_time - last_time
    last_time = cur_time
    tot_time = cur_time - begin_time

    L = []
    L.append('  Step: %s' % format_time(step_time))
    L.append(' | Tot: %s' % format_time(tot_time))
    if msg:
        L.append(' | ' + msg)

    msg = ''.join(L)
    sys.stdout.write(msg)
    for i in range(term_width-int(TOTAL_BAR_LENGTH)-len(msg)-3):
        sys.stdout.write(' ')

    # Go back to the center of the bar.
    for i in range(term_width-int(TOTAL_BAR_LENGTH/2)+2):
        sys.stdout.write('\b')
    sys.stdout.write(' %d/%d ' % (current+1, total))

    if current < total-1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()

def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds*1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f

In [ ]:
# parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
# parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
# parser.add_argument('--resume', '-r', action='store_true',
#                     help='resume from checkpoint')
# args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

transform_train = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

data_dir = '/content/image'
dataset_train = datasets.ImageFolder(data_dir, transform=transform_train)
dataset_test = datasets.ImageFolder(data_dir, transform=transform_test)

dataset_sizes = len(dataset_train)
classes = dataset_train.classes
print(classes)

def divide_treino_validacao(train_rate=0.9):
    """
    As classes estao divididas no dataset da seguinte forma: (0-99: classe 1),
                                                             (100-199: classe 2),
                                                             ...
                                                             (900-999: classe 10)
    
    Temos que dividir os indices aleatoriamente para treino e validacao, mas lembrando que as 
    proporcoes definidas para treino e validacao devem ser respeitadas entre cada uma das classes.
    """
    train_indices = []
    val_indices = []
    for i in range(10):
        ind = np.arange(i*100, (i+1)*100)
        ind = np.random.permutation(ind)
        
        frontier = int(len(ind)*train_rate)
        ind_t = ind[0:frontier]
        ind_v = ind[frontier:len(ind)]

        train_indices += ind_t.tolist()
        val_indices += ind_v.tolist()

    return (train_indices, val_indices)
        
train_indices, val_indices = divide_treino_validacao(0.5)

trainset = torch.utils.data.Subset(dataset_train, train_indices)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)

testset = torch.utils.data.Subset(dataset_test, val_indices)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

# trainset = torchvision.datasets.CIFAR10(
#     root='./data', train=True, download=True, transform=transform_train)
# trainloader = torch.utils.data.DataLoader(
#     trainset, batch_size=128, shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(
#     root='./data', train=False, download=True, transform=transform_test)
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=100, shuffle=False, num_workers=2)

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = resnet18(pretrained=False)
net = MixedNet()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()
# net = SimpleDLA()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# if args.resume:
#     # Load checkpoint.
#     print('==> Resuming from checkpoint..')
#     assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
#     checkpoint = torch.load('./checkpoint/ckpt.pth')
#     net.load_state_dict(checkpoint['net'])
#     best_acc = checkpoint['acc']
#     start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc, state
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc >= best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        # if not os.path.isdir('checkpoint'):
        #     os.mkdir('checkpoint')
        # torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


# for epoch in range(start_epoch, start_epoch+2):
#     train(epoch)
#     test(epoch)
#     scheduler.step()

['fabric', 'foliage', 'glass', 'leather', 'metal', 'paper', 'plastic', 'stone', 'water', 'wood']
==> Building model..


In [ ]:
#Avaliando na FMD a LBCNN sem estar treinada
test(0)

 [===============================================================>.]  Step: 462ms | Tot: 55s119ms | Loss: 14092022024220346.000 | Acc: 8.200% (41/500) 63/63 
Saving..


In [ ]:
for epoch in range(start_epoch, start_epoch+50):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
 [===============================================================>.]  Step: 883ms | Tot: 1m43s | Loss: 2.949 | Acc: 16.600% (83/500) 63/63 
 [===============================================================>.]  Step: 468ms | Tot: 55s404ms | Loss: 1.885 | Acc: 32.200% (161/500) 63/63 
Saving..

Epoch: 1
 [===============================================================>.]  Step: 851ms | Tot: 1m41s | Loss: 1.915 | Acc: 38.200% (191/500) 63/63 
 [===============================================================>.]  Step: 463ms | Tot: 55s310ms | Loss: 1.656 | Acc: 46.800% (234/500) 63/63 
Saving..

Epoch: 2
 [===============================================================>.]  Step: 908ms | Tot: 1m43s | Loss: 1.648 | Acc: 44.200% (221/500) 63/63 
 [===============================================================>.]  Step: 477ms | Tot: 56s645ms | Loss: 1.304 | Acc: 61.000% (305/500) 63/63 
Saving..

Epoch: 3
 [===============================================================>.

In [ ]:
state['acc'], state['epoch']

(74.0, 17)

In [ ]:
# torch.save(state['net'], '/content/drive/MyDrive/IC/ArquiteturaBifurcada1.pt')